# Import Dependency

In [18]:
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras

# Regression

In [2]:
# Import data
df_reg = pd.read_csv('FlightPricePrediction.csv')
df_reg

,Airline,Source,Destination,Total_Stops,Additional_Info,Duration_Hours,Price
0,IndiGo,Banglore,New Delhi,0,No info,2.833333,3897
1,Air India,Kolkata,Banglore,2,No info,7.416667,7662
2,Jet Airways,Delhi,Cochin,2,No info,19.000000,13882
3,IndiGo,Kolkata,Banglore,1,No info,5.416667,6218
4,IndiGo,Banglore,New Delhi,1,No info,4.750000,13302
...,...,...,...,...,...,...,...
10676,Air Asia,Kolkata,Banglore,0,No info,2.500000,4107
10677,Air India,Kolkata,Banglore,0,No info,2.583333,4145
10678,Jet Airways,Banglore,Delhi,0,No info,3.000000,7229
10679,Vistara,Banglore,New Delhi,0,No info,2.666667,12648


In [3]:
# One-hot encoding pada data nominal
df_reg = pd.get_dummies(df_reg, drop_first=True)

In [4]:
# Memisahkan features and label
X_reg = df_reg.drop('Price', axis=1)
y_reg = df_reg['Price']

In [5]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_reg, y_reg, test_size=0.2, random_state=42)

# Normalize the features using StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [6]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train_scaled.shape[1],)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

# Compile the model
model.compile(optimizer='adam', loss='mean_absolute_error')

In [7]:
model.fit(X_train_scaled, y_train, epochs=100, batch_size=64, validation_split=0.2)

Epoch 1/100
107/107 [==============================] - 7s 10ms/step - loss: 9074.7891 - val_loss: 8993.7207
Epoch 2/100
107/107 [==============================] - 1s 8ms/step - loss: 8736.6973 - val_loss: 8261.3643
Epoch 3/100
107/107 [==============================] - 1s 6ms/step - loss: 7352.8164 - val_loss: 6148.7578
Epoch 4/100
107/107 [==============================] - 1s 6ms/step - loss: 4720.6494 - val_loss: 3224.8552
Epoch 5/100
107/107 [==============================] - 1s 6ms/step - loss: 2720.2529 - val_loss: 2343.9548
Epoch 6/100
107/107 [==============================] - 1s 6ms/step - loss: 2221.5552 - val_loss: 2062.5725
Epoch 7/100
107/107 [==============================] - 1s 8ms/step - loss: 1985.9227 - val_loss: 1899.4590
Epoch 8/100
107/107 [==============================] - 1s 7ms/step - loss: 1851.5414 - val_loss: 1799.3329
Epoch 9/100
107/107 [==============================] - 1s 8ms/step - loss: 1777.8223 - val_loss: 1751.0116
Epoch 10/100
107/107 [==============

In [8]:
loss = model.evaluate(X_test_scaled, y_test)
print(f"Mean Absolute Error on Test Data: {loss}")

67/67 [==============================] - 0s 2ms/step - loss: 1551.6803
Mean Absolute Error on Test Data: 1551.6802978515625


In [9]:
# Create a more complex neural network model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train_scaled.shape[1],)),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

# Compile the model
model.compile(optimizer='adam', loss='mean_absolute_error')

# Train the model
model.fit(X_train_scaled, y_train, epochs=100, batch_size=32, validation_split=0.2)

# Evaluate the model
loss = model.evaluate(X_test_scaled, y_test)
print(f"Mean Absolute Error on Test Data: {loss}")

Epoch 1/100
214/214 [==============================] - 2s 4ms/step - loss: 5471.8296 - val_loss: 1885.0376
Epoch 2/100
214/214 [==============================] - 1s 4ms/step - loss: 1769.1608 - val_loss: 1678.9130
Epoch 3/100
214/214 [==============================] - 1s 4ms/step - loss: 1682.3716 - val_loss: 1666.8274
Epoch 4/100
214/214 [==============================] - 1s 4ms/step - loss: 1657.5067 - val_loss: 1615.3481
Epoch 5/100
214/214 [==============================] - 1s 5ms/step - loss: 1629.6399 - val_loss: 1607.0287
Epoch 6/100
214/214 [==============================] - 1s 6ms/step - loss: 1610.6086 - val_loss: 1565.0192
Epoch 7/100
214/214 [==============================] - 1s 4ms/step - loss: 1584.2866 - val_loss: 1558.1528
Epoch 8/100
214/214 [==============================] - 1s 4ms/step - loss: 1578.1481 - val_loss: 1547.0383
Epoch 9/100
214/214 [==============================] - 1s 4ms/step - loss: 1566.1165 - val_loss: 1576.6469
Epoch 10/100
214/214 [===============

# Classification

In [16]:
# Import data
df_cls = pd.read_csv('gender.csv')
df_cls

,long_hair,forehead_width_cm,forehead_height_cm,nose_wide,nose_long,lips_thin,distance_nose_to_lip_long,gender
0,1,11.8,6.1,1,0,1,1,Male
1,0,14.0,5.4,0,0,1,0,Female
2,0,11.8,6.3,1,1,1,1,Male
3,0,14.4,6.1,0,1,1,1,Male
4,1,13.5,5.9,0,0,0,0,Female
...,...,...,...,...,...,...,...,...
4996,1,13.6,5.1,0,0,0,0,Female
4997,1,11.9,5.4,0,0,0,0,Female
4998,1,12.9,5.7,0,0,0,0,Female
4999,1,13.2,6.2,0,0,0,0,Female


In [19]:
label_encoder = LabelEncoder()
df_cls['gender'] = label_encoder.fit_transform(df_cls['gender'])

In [20]:
# Memisahkan features and label
X_cls = df_cls.drop('gender', axis=1)
y_cls = df_cls['gender']

In [21]:
# Membagi dataset menjadi training set dan test set
# dengan jumlah test set adalah sebanyak 20% dari data keseluruhan
X_train, X_test, y_train, y_test = train_test_split(X_cls, y_cls, test_size=0.2, random_state=42)

In [22]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(16, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Binary classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/50
125/125 [==============================] - 3s 7ms/step - loss: 0.7036 - accuracy: 0.7757 - val_loss: 0.4904 - val_accuracy: 0.9421
Epoch 2/50
125/125 [==============================] - 0s 3ms/step - loss: 0.4064 - accuracy: 0.9438 - val_loss: 0.3278 - val_accuracy: 0.9441
Epoch 3/50
125/125 [==============================] - 0s 4ms/step - loss: 0.2410 - accuracy: 0.9507 - val_loss: 0.1861 - val_accuracy: 0.9481
Epoch 4/50
125/125 [==============================] - 0s 3ms/step - loss: 0.1443 - accuracy: 0.9535 - val_loss: 0.1477 - val_accuracy: 0.9510
Epoch 5/50
125/125 [==============================] - 0s 4ms/step - loss: 0.1183 - accuracy: 0.9530 - val_loss: 0.1177 - val_accuracy: 0.9461
Epoch 6/50
125/125 [==============================] - 0s 3ms/step - loss: 0.1074 - accuracy: 0.9525 - val_loss: 0.1112 - val_accuracy: 0.9461
Epoch 7/50
125/125 [==============================] - 0s 3ms/step - loss: 0.1034 - accuracy: 0.9498 - val_loss: 0.1137 - val_accuracy: 0.9500
Epoch 

In [23]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_acc:.4f}')

32/32 [==============================] - 0s 2ms/step - loss: 0.1012 - accuracy: 0.9570
Test accuracy: 0.9570
